**TOPIC: a**<br>
**Questions**<br>
**1. What data do you have?**<br>
answer<br><br>
**2. What would you like to know?**<br>
answer<br><br>
**3. Explore the data.**<br>
Generate statistics and perform visualizations. Explain what you are computing (mean, SD, ...), and then
compute using Python. Create some visualizations (at least 8, and at least 6
must be of different types); use Python.<br><br>
**4. Can you state any hypotheses or make predictions? Which tests can you apply to
verify your hypothesis? State clearly each of your hypotheses (at least 3).**<br>
answer<br><br>
**5. Test your hypotheses.**<br>
Test your hypotheses and predictions (use at least 2 different
tests). For each: i) describe the test you are using; ii) perform it; iii) analyze
the results and draw the conclusion. You must perform correlation analysis and chi-squared test.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import chi2_contingency
from scipy.stats import chi2
df = pd.read_csv("Responses.csv")

In [2]:
df = df[['1. How old are you?',
         '2. To which gender identity do you most identify?',
         '62. On average, how many hours do you listen to music per day?', 
         '68. Which social media platforms do you use most often?', 
         '69. What is your favorite genre of music?', 
         '70. In which of the following situations do you listen to music?', 
         '71. Which music streaming service do you use the most?']]
df = df.rename(columns={'1. How old are you?': 'Age', 
                        '2. To which gender identity do you most identify?': 'Gender', 
                        '62. On average, how many hours do you listen to music per day?': 'Daily_Music_Consumption', 
                        '68. Which social media platforms do you use most often?': 'Preferred_Social_Medias', 
                        '69. What is your favorite genre of music?': 'Music_Genre', 
                        '70. In which of the following situations do you listen to music?': 'Occasion', 
                        '71. Which music streaming service do you use the most?': 'Preferred_Music_Platform'})
df.loc[~df['Gender'].isin(['Man', 'Woman']), 'Gender'] = 'Other'
df.loc[df['Music_Genre'] == "Breakcore", 'Music_Genre'] = "Electronic"
df.loc[df['Music_Genre'] == "Bladee", 'Music_Genre'] = "Hip Hop / Rap"
df.loc[df['Music_Genre'] == "Funk", 'Music_Genre'] = "Pop"
df.loc[df['Music_Genre'] == "House", 'Music_Genre'] = "Electronic"
df.loc[df['Music_Genre'] == "I have no favorite", 'Music_Genre'] = "No preference"
df.loc[df['Music_Genre'] == "Lofi", 'Music_Genre'] = "Other"

KeyError: "['2. To which gender identity do you most identify?'] not in index"

In [ ]:
df.loc[df['Music_Genre'] == "Most genres like anime, jpop, edm, hip hop, kpop, pop, r&b, lofi", 'Music_Genre'] = "Anime / Jpop"
df.loc[df['Music_Genre'] == "Musical Theater", 'Music_Genre'] = "Other"
df.loc[df['Music_Genre'] == "Phonk", 'Music_Genre'] = "Electronic"
df.loc[df['Music_Genre'] == "Salsa music", 'Music_Genre'] = "Other"
df.loc[df['Music_Genre'] == "Soundtrack ", 'Music_Genre'] = "Other"
df.loc[df['Music_Genre'] == "Video Game", 'Music_Genre'] = "Other"
df.loc[df['Music_Genre'] == "any, whatever sounds good to my ear", 'Music_Genre'] = "No preference"
df.loc[df['Music_Genre'] == "electronic pop ", 'Music_Genre'] = "Electronic"
df.loc[df['Music_Genre'] == "idk", 'Music_Genre'] = "No preference"
df.loc[df['Music_Genre'] == "indie pop", 'Music_Genre'] = "Indie / Alternative"
df.loc[df['Music_Genre'] == "multi", 'Music_Genre'] = "Other"
df = df[df['Daily_Music_Consumption'] <= 24]
def categorize_hours(hours):
    if hours <= 2:
        return '0-2'
    elif hours <= 5:
        return '3-5'
    elif hours <= 8:
        return '6-8'
    else:
        return '8+'

df['Hours_Cohort'] = df['Daily_Music_Consumption'].apply(categorize_hours)

def categorize_age(ages):
    if ages <= 20:
        return '19 to 20'
    elif ages <= 22:
        return '21 to 22'
    elif ages <= 24:
        return '23 to 24'
    else:
        return '24+'
df['Age_Cohort'] = df['Age'].apply(categorize_age)

In [ ]:
# nab all Reddit users
sm = df[df["Preferred_Social_Medias"].str.contains("Reddit", na=False)]
sm.reset_index()
print(sm.groupby("Music_Genre")["Age"].count().plot.bar())
#######
##Breakdown of Prefered Genre by Gender(notice Rap vs Indie )
table_4 = pd.pivot_table(data = df, values = 'Age', columns = 'Music_Genre', index = 'Gender', aggfunc = 'count' )
table_4.loc [ [ 'Man' , 'Woman', 'Other'  ] ].plot(kind = 'bar')

##Vast majority of responses contain Instagram 
tmp = df['Preferred_Social_Medias'].astype("string").str[0];

tmp.value_counts().nlargest(7)

In [ ]:
##Getting rid of redundant platforms, less than 5 people chose the omitted platforms 
df2=df.query("Preferred_Music_Platform != ('Amazon Prime Music', 'Soundcloud', 'Pandora')")
##Music Platform vs Genre, probably nothing here 
table_5 = pd.pivot_table(data = df2, values = 'Age', columns = 'Preferred_Music_Platform', index = 'Music_Genre', aggfunc = 'count' )
table_5.loc [ [ 'Electronic' , 'Hip Hop / Rap' , 'Kpop' , 'Metal', 'No preference', 'Pop' , 'Rock', 'Indie / Alternative', 'Anime / Jpop',    ] ].plot(kind = 'bar')
##Age vs Genre preferrence 
table_6 = pd.pivot_table(data = df, values = 'Age', columns = 'Age_Cohort', index = 'Music_Genre', aggfunc = 'count' )
table_6.loc [ [ 'Electronic' , 'Hip Hop / Rap' , 'Kpop' , 'Metal', 'No preference', 'Pop' , 'Rock', 'Indie / Alternative', 'Anime / Jpop',    ] ].plot(kind = 'bar')

**ANALYSIS OF AGE AND HOURS SPENT LISTENING TO MUSIC**<br>

**Below is a chart which pairs age with hours spent listening to music. At a glance, it seems to demonstrate a negative correlation between the two variables. Notice that the oldest cohort does not report listening to more than 8 hours of music per day. It seems that the older a respondant is, the lower their daily music consumption will be.**

In [ ]:
##Hours vs age cohort, there is a slight negative correlation between Age and Hours
table_7 = pd.pivot_table(data = df, values = 'Age', columns = 'Age_Cohort', index = 'Hours_Cohort', aggfunc = 'count' )
table_7.plot(kind = 'bar')

**ANALYSIS OF AGE AND HOURS SPENT LISTENING TO MUSIC**<br>

**We can show the raw numbers better in a pivot table hilighting the two varibles. It seems here as well that the older repondants reported less music consumption per day. We concede that the number of older repondants is dwarfed by the younger cohorts. In the following cells, we take the joint probability and visualize it on a heat map to gain further insight.**

In [ ]:
Hours_Age_counts = df.pivot_table(
    index="Age_Cohort", columns="Hours_Cohort",
    values="Age",  # We can pretty much count any column, as long as there are no NaNs.
    aggfunc="count", # The count function will count the number of non-null values.
    fill_value=0
    
)
Hours_Age_counts

In [ ]:
print(Hours_Age_counts.sum().sum())
joint = Hours_Age_counts / Hours_Age_counts.sum().sum()
joint

In [ ]:
joint.sum(axis=1)

In [ ]:
joint.sum(axis=0)

In [ ]:
pd.crosstab(df.Age_Cohort, df.Hours_Cohort,
            normalize=True, margins=True)

**ANALYSIS OF AGE AND HOURS SPENT LISTENING TO MUSIC**<br>

**Below we have a heat map of the joint probability. Note that given a random survey respondant, they are most likely a 19-20 year old who listens to 0-2 hours of music per day. That is to say, they are the largest subgroup of respondants shown on the heat map. Generally, it seems that you have a very low probability of being in the older cohort and listening to alot of music.**

In [ ]:
import seaborn as sns

sns.heatmap(joint)

**ANALYSIS OF AGE AND HOURS SPENT LISTENING TO MUSIC**<br>

**Here we examine the conditional probability between the hours and age cohorts. We calculate the probability of one variable given the other and represent that data with a stacked bar graph.**

In [ ]:
Hours_counts = Hours_Age_counts.sum(axis=1)
Hours_Given_Age = Hours_Age_counts.divide(Hours_counts, axis=0)
Hours_Given_Age

**ANALYSIS OF AGE AND HOURS SPENT LISTENING TO MUSIC**<br>

**Here it seems abundantly clear that older respondants spend less time listening to music. There is a zero percent chance that a person 23+ is listening to more than 8 hours of music. We also see that the older cohorts are more likley to report the lowest number of hours listening to music.**

In [ ]:
Hours_Given_Age.plot.bar(stacked=True)

**ANALYSIS OF AGE AND HOURS SPENT LISTENING TO MUSIC**<br>

**Our initial exploration of the data seems to indicate that these two variables are related. Our hypothesis is that music consumption is negativley correlated with the age of the survey respondant. We will test this hypotheis using the pearson correlation test and the chi-squared test.**

In [ ]:
# calculate the pearson's correlation between Age and Music Consumption
df.Daily_Music_Consumption.corr(df.Age, method = 'pearson')

**ANALYSIS OF AGE AND HOURS SPENT LISTENING TO MUSIC**<br>

**The pearson correlation coefficient (r) is about -0.145, indicating a weak negative correlation between age and music consumption. This does not look brilliant for our hypothesis but a weak corelation is still indicated. The chi squared test will indicate if our findings are statistically relevant.**

In [ ]:
Hours_Age_counts

**ANALYSIS OF AGE AND HOURS SPENT LISTENING TO MUSIC**<br>

**Our null hypothesis is that Age and Music Consumption are independent. With 4 rows and 4 columns in our dataset, we have that (R-1)(C-1) = 9 degrees of freedom. We have a p vlue of 0.1585 meaning that this chi squared value would occurr by chance about 15% of the time. Convention states that we should use a significance level of 0.05 to determine the critical value. With 9 degrees of freedom, our critical value is 16.919. Since our chi squared value is 13.09, and less than the critical value, we cannot reject the null hypothesis. Based on this test, these two variables are independent.**

In [ ]:
from scipy.stats import chi2_contingency
chi, p, dfree, expected = chi2_contingency(Hours_Age_counts)
print("calculated chi square value: " ,chi)
print("calculated p value: " ,p)
print("degree of freedom: ", dfree)
##So We need an alpha value of 0.1585 to reject the null hypothesis. Since convention seems to indicate that
#an alpha of 0.05 should be used, We cannot reject the null hypothesis, meaning that the variables are independent. 

In [ ]:
##Heat map
r=df.corr()
r

sns.heatmap(r, center= 0, cmap = "coolwarm",annot= True)
plt.show()

In [ ]:
Medias_Genre_counts = df.pivot_table(
    index="Preferred_Social_Medias", columns="Music_Genre",
    values="Age",  # We can pretty much count any column, as long as there are no NaNs.
    aggfunc="count" # The count function will count the number of non-null values.
)
Medias_Genre_counts

In [ ]:
counts = pd.crosstab(df.Preferred_Social_Medias, df.Music_Genre)
counts
#print(counts.sum().sum())
#joint = counts / counts.sum().sum()
#joint

In [ ]:
print(counts.sum().sum())
joint = counts / counts.sum().sum()
joint